# 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## 데이터 불러오기
train = pd.read_csv('/content/drive/My Drive/open_data/train.csv')
test = pd.read_csv('/content/drive/My Drive/open_data/test.csv')
submission = pd.read_csv('/content/drive/My Drive/open_data/sample_submission.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36325 entries, 0 to 36324
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          36325 non-null  int64 
 1   Q1             36325 non-null  int64 
 2   Q2             36325 non-null  int64 
 3   Q3             36325 non-null  int64 
 4   Q4             36325 non-null  int64 
 5   Q5             36325 non-null  int64 
 6   Q6             36325 non-null  int64 
 7   Q7             36325 non-null  int64 
 8   Q8             36325 non-null  int64 
 9   Q9             36325 non-null  int64 
 10  Q10            36325 non-null  int64 
 11  Q11            36325 non-null  int64 
 12  Q12            36325 non-null  int64 
 13  Q13            36325 non-null  int64 
 14  Q14            36325 non-null  int64 
 15  Q15            36325 non-null  int64 
 16  Q16            36325 non-null  int64 
 17  Q17            36325 non-null  int64 
 18  Q18            36325 non-n

## 데이터 전처리

In [ ]:
target = train['nerdiness']
train.drop(['nerdiness'], axis=1, inplace=True)
all_data = pd.concat([train, test], axis=0)
print(all_data.shape)
all_data.head()

(50452, 80)


,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,screenw,screenh,hand,religion,orientation,voted,married,familysize,major,race_arab,race_asian,race_black,race_white,race_hispanic,race_nativeam,race_nativeau,race_other,ASD
0,0,4,2,4,1,2,4,4,4,2,4,5,4,4,2,2,1,4,4,4,3,1,1,2,3,3,4,US,6,150,190,2,5,3,5,6,6,3,5,5,5,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,2,2,2,1,18,360,640,1,3,1,1,1,3,NaN,0,1,0,0,0,0,0,0,2
1,1,5,5,4,5,3,4,5,5,4,4,5,4,4,5,5,4,5,4,1,5,3,3,4,5,2,4,IT,3,145,115,5,7,6,6,7,6,4,1,4,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,3,3,2,2,27,360,640,1,1,1,1,1,2,Law,0,0,0,1,0,0,0,0,2
2,2,3,3,4,4,5,4,5,2,5,4,2,4,3,4,1,2,1,5,2,3,1,2,4,5,2,5,US,2,87,152,5,2,7,5,7,5,7,6,6,2,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,4,3,1,1,37,1024,1366,1,1,1,1,3,1,Computer science; religion,0,0,0,1,0,0,1,0,2
3,3,4,5,3,5,3,4,5,2,5,2,1,3,2,4,3,1,1,5,1,3,1,1,4,5,5,5,GB,67,73,117,6,1,6,3,7,3,5,3,6,2,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,4,3,2,1,32,1440,900,2,2,1,1,1,1,Politics and Philosophy,0,0,0,1,0,0,0,0,2
4,4,3,5,4,3,3,3,4,4,4,3,4,2,4,3,4,5,4,5,1,2,4,1,4,3,2,3,CA,6,86,118,1,4,6,5,6,5,5,4,4,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,3,2,1,1,47,1920,1080,1,2,1,1,2,2,Business,0,0,0,1,0,0,0,0,2


In [ ]:
all_data.loc[all_data['gender'] == 0, 'gender'] = 2
all_data.loc[all_data['married'] == 0, 'married'] = 1
all_data.loc[all_data['ASD'] == 0, 'ASD'] = 2
all_data.loc[all_data['education'] == 0, 'education'] =\
    all_data['education'].mode()[0]
all_data.loc[all_data['voted'] == 0, 'voted'] =\
    all_data['voted'].mode()[0]
all_data.loc[all_data['urban'] == 0, 'urban'] =\
    all_data['urban'].mode()[0]
all_data.loc[all_data['orientation'] == 0, 'orientation'] =\
    all_data['orientation'].mode()[0]
all_data.loc[all_data['religion'] == 0, 'religion'] =\
    all_data['religion'].mode()[0]
all_data.loc[all_data['engnat'] == 0, 'engnat'] =\
    all_data['engnat'].mode()[0]
all_data.loc[all_data['familysize'] > 7, 'familysize'] =\
    all_data['familysize'].mode()[0]
all_data.loc[all_data['age'] > 6, 'age'] = all_data['familysize'].mode()[0]


train = all_data.iloc[:train.shape[0]]
test = all_data.iloc[train.shape[0]:]

In [ ]:
country_map = {i: c for i, c in enumerate(train['country'].unique())}
major_map = {i: c for i, c in enumerate(train['major'].unique())}


train['country'] = train['country'].map(country_map)
test['country'] = test['country'].map(country_map)

train['major'] = train['major'].map(major_map)
test['major'] = test['major'].map(major_map)

train = train.fillna(0)
test = test.fillna(0)

In [ ]:
x_train = train.loc[:, 'Q1':'ASD']
y_train = target

x_test = test.loc[:, 'Q1':'ASD']



X_train, X_test, y_train, y_test =\
    train_test_split(train, target, test_size=0.3, random_state=2020)

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, rf_pred)

0.9392790118618555

In [ ]:
lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train, y_train)
lgbm_pred = lgbm_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, lgbm_pred)

0.8383629524443555

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=713)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, rf_pred)

0.9425038615799847

In [ ]:
# lgbm 분류기
lgbm_clf = LGBMClassifier(
            n_estimators=1000,
            learning_rate=0.02,
            random_state=94,
            max_depth=11,
            num_leaves=573,
            min_child_samples=10,
            subsample=0.76,
            colsample_bytree=0.52,
            max_bin=449,
            reg_lambda=0.07,
            reg_alpha=0.28
        )
lgbm_clf.fit(X_train, y_train)
lgbm_pred = lgbm_clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, lgbm_pred)

0.9353186719007063

In [ ]:
y_preds = 0.12* lgbm_pred + 0.88 * rf_pred
roc_auc_score(y_test, y_preds)

0.9428315014741233

In [16]:
lgbm_clf.fit(train, target)
lgbm_pred = lgbm_clf.predict_proba(test)[:, 1]
rf_clf.fit(train, target)
rf_pred = rf_clf.predict_proba(test)[:, 1]
y_pred =  0.2* lgbm_pred + 0.8 *rf_pred
submission['nerdiness'] = y_pred
submission.to_csv('weight_ensemble.csv', index=False)